# Plotting image data on a 3D Earth Model

In [ ]:
import sys
import accessvis
import lavavu
import numpy as np
import math
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import xarray as xr

## Select our model resolution

This widget gives a quick way of choosing the level of detail to load, lower detail is faster and uses less memory but doesn't look as good, higher levels will be so slow to render that it will be difficult to interact with the model, best left for final renders.

The default parameter sets the initial choice [1-4]

In [ ]:
accessvis.resolution_selection(default=1)

## Create a simple earth relief plot

- See previous tutorial for details

In [ ]:
lv = accessvis.plot_earth(texture='bluemarble', background="white", vertical_exaggeration=20)

#Zoom out
lv.translation(0.0, 0.0, -17)
lv.rotation(0.0, -125.0, 0.0) #Rotate to Australia

In [ ]:
#Update lighting
#lv.set_properties(diffuse=0.8, ambient=0.1, specular=0.35, shininess=0.03, light=[1,1,0.98])

In [ ]:
#Increase the colour saturation a bit
#lv.brightness_contrast_saturation(0.5, 0.5, 0.65)

In [ ]:
lv.display((400,400))

## Plot image data
Worldview snapshots https://wvs.earthdata.nasa.gov/

In [ ]:
import time
baseurl = 'https://wvs.earthdata.nasa.gov/api/v1/snapshot?'
params = {'REQUEST': 'GetSnapshot',
          'LAYERS': 'VIIRS_NOAA20_CorrectedReflectance_TrueColor',
          'CRS': 'EPSG:4326',
          'TIME': '2024-09-25',
          'WRAP': 'DAY',
          'BBOX': '-90,-180,90,180',
          'FORMAT': 'image/jpeg',
          'WIDTH': 8192,
          'HEIGHT': 4096,
          'AUTOSCALE': 'TRUE',
          'ts': int(time.time()*1000)
         }
plist = [f"{p}={params[p]}" for p in params]
url = baseurl + '&'.join(plist)
url

fn = 'worldview_earth_snapshot.jpg'

In [ ]:
if not os.path.exists(fn):
    accessvis.download(url, filename=fn)

In [ ]:
#Rotate the texture 180 degrees and set alpha channel in nodata regions
img = Image.open(fn)
arr = np.array(img)
arr = np.roll(arr, arr.shape[1] // 2, axis=1)
plt.imshow(arr)

In [ ]:
#Use approximate luminance for opacity (so darker = transparent)
#This will reduce the data to mostly clouds and ice areas to overlay over our existing texture
alpha = (0.33*arr[:,:,0] + 0.5*arr[:,:,1] + 0.16*arr[:,:,2]).astype(np.uint8)

In [ ]:
alpha.shape

In [ ]:
arr.shape

In [ ]:
arr2 = np.dstack((arr, alpha))
plt.imshow(arr2)

# Add image data


In [ ]:
accessvis.update_earth_values(lv, dataMode=0, data=arr2)

In [ ]:
lv.reset()

In [ ]:
lv.display((400,400))

In [ ]:
lv.window((600,600))